In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
n =  8
rng = np.random.default_rng(seed=0)
df = pd.DataFrame(np.around(rng.random((n, 4)), 1), columns=['a','b','c','y'])

In [3]:
# The way the model will go through the values of the df
OUT_STEPS = 2  
input_width = 2   # Take two rows (or time steps) of all columns as input
label_width = OUT_STEPS  # Size of the prediction (output)
shift = OUT_STEPS  # Time (or rows) offset between input and output
total_window_size = input_width + shift
batch_size = 1
label_index = None #In the future will be the index of 'y'

In [4]:
# Just a sort of conversion from an array to an tf._.EagerTensor
data = np.array(df.values, dtype=np.float32)
def stack_data(data, total_window_size):
    batches = []
    start = 0
    end = total_window_size
    for start in range(data.shape[0]-1):
        batch = data[start:end]
        start = start + total_window_size + 1
        end = start
        if batch.shape[0] == total_window_size:
            batches.append(batch)
    return tf.stack(batches)
stacked_data = stack_data(data, total_window_size)

I want to calculate a baseline model. In this case, a model that averages the last values of the input, and repeats it OUT_STEPS times in the target

In [5]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

In [6]:
class MultiStepMeanBaseline(tf.keras.Model):
    """
    A mixture of MultiStepLastBaseline & RepeatBaseline in the link below 
    but calculating the average instead of returning the last value.
    https://www.tensorflow.org/tutorials/structured_data/time_series#baselines
    """
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index


    def call(self, inputs): 
        """
        Calls the model on new inputs.
        Returns the outputs as tensors.
        https://www.tensorflow.org/api_docs/python/tf/keras/Model#call
        """
        # inputs.shape: (None, 2, 4) (batch, time_steps <rows>, features <columns>)
        # type(inputs): <class 'tensorflow.python.framework.ops.Tensor'>

        if self.label_index is None:
            # How can I grab each input & average the values along the time?
            average_time_dim = inputs[:, np.mean(:), :] # SYNTAXERROR
            # possible shape of average_time_dim: (4,)
            average_reshaped = average_time_dim[tf.newaxis, tf.newaxis, :]
            return tf.tile(average_reshaped, [1, self.out_steps, 1])
        # TODO.
        # result = inputs[:, :, self.label_index]
        # return result[:, :, tf.newaxis]



According to the tutorial, the data is splitted into inputs and outputs with a function similar to 

In [7]:
input_slice = slice(0, input_width)
label_slice = slice(total_window_size-label_width, None)
def split_stacked_data(stacked_data):
    """
    Split dataset into inputs and labels (or targets)
    https://www.tensorflow.org/tutorials/structured_data/time_series#2_split
    """
    inputs = stacked_data[:,input_slice, :] 
    labels = stacked_data[:,label_slice,label_index:] 
    inputs.set_shape([None, input_width, None])
    labels.set_shape([None, label_width, None])
    return inputs, labels
# inputs, labels = split_stacked_data(stacked_data)

And then, the dataset is created, also, the split function is mapped to the dataset. 

In [8]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/timeseries_dataset_from_array
input_dataset = tf.keras.utils.timeseries_dataset_from_array(
    data=data,              # all columns of the df from 1st to penultimate rows
    targets=None,           # To deliver a single dataset
    sequence_length=total_window_size,   # Yields 1 step per batch in the (input) data
    sequence_stride=1,                  # Shifts 1 step for next batches
    shuffle=False,      
    batch_size=batch_size  # Works only with 1 but the loss & error are the same as in the timeseries tutorial
    )
input_dataset = input_dataset.map(split_stacked_data)

The resulting dataset has inputs of shape `(batch_zise, input_width, df.shape[1])` and the outputs have a shape of `(batch_size, OUT_STEPS, 1)`

In [9]:
for batch in input_dataset.take(2):
    input, target = batch
    print(f'input: {input}')
    print(f'target: {target}')
print(f'input.shape: {input.shape}')
print(f'target.shape: {target.shape}')

input: [[[0.6 0.3 0.  0. ]
  [0.8 0.9 0.6 0.7]]]
target: [[[0.5 0.9 0.8 0. ]
  [0.9 0.  0.7 0.2]]]
input: [[[0.8 0.9 0.6 0.7]
  [0.5 0.9 0.8 0. ]]]
target: [[[0.9 0.  0.7 0.2]
  [0.9 0.5 0.3 0.4]]]
input.shape: (1, 2, 4)
target.shape: (1, 2, 4)


In [10]:
# Baseline from WindowGenerator
baseline_model = RepeatBaseline()
baseline_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()]) # MAE
# Evaluation from WindowGenerator
evaluation = baseline_model.evaluate(input_dataset, verbose=0)
print(evaluation)

[0.2344999760389328, 0.39500001072883606]


In [11]:
baseline_model.predict(input_dataset)

5/5 [==============================] - 0s 2ms/step


array([[[0.6, 0.3, 0. , 0. ],
        [0.8, 0.9, 0.6, 0.7]],

       [[0.8, 0.9, 0.6, 0.7],
        [0.5, 0.9, 0.8, 0. ]],

       [[0.5, 0.9, 0.8, 0. ],
        [0.9, 0. , 0.7, 0.2]],

       [[0.9, 0. , 0.7, 0.2],
        [0.9, 0.5, 0.3, 0.4]],

       [[0.9, 0.5, 0.3, 0.4],
        [0. , 0.1, 0.7, 0.6]]], dtype=float32)

In [ ]:
# array([[[0.7, 0.6, 0.3, 0.35],
#         [0.7, 0.6, 0.3, 0.35]],

#        [[0.65, 0.9, 0.7, 0.35],
#         [0.65, 0.9, 0.7, 0.35]],

#        [[0.7, 0.45, 0.75, 0.1],
#         [0.7, 0.45, 0.75, 0.1]],
# ...                           ]]